### Query rewrite

In [ ]:
from openai import OpenAI
import os

def llm_query_rewrite(original_query):
    client = OpenAI()
    
    prompt = f"""
    원본 쿼리: {original_query}
    
    위 쿼리를 다음 기준으로 다시 작성해주세요:
    1. 모호함 제거
    2. 관련 키워드 추가
    3. 검색에 효과적인 형태로 변환
    
    다시 작성된 쿼리만 출력하세요.
    """
    
    response = client.chat.completions.create(
        model=os.getenv("OPENAI_DEFAULT_MODEL"),
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
    )
    
    return response.choices[0].message.content.strip()

query = "인공지능이란 무엇인가?"
print(llm_query_rewrite(query))

In [ ]:
def hyperfocus_rewrite(query, context=None):
    """
    여러 관점에서 쿼리를 재작성하는 하이퍼포커스 기법
    """
    client = OpenAI()
    
    prompt = f"""
    원본 쿼리: {query}
    
    이 쿼리에 대한 검색 결과를 최적화하기 위해 다음 5가지 관점에서 쿼리를 재작성해주세요:
    1. 정의 관점: 개념이나 용어의 정의를 찾는 쿼리
    2. 방법 관점: 특정 작업의 수행 방법을 찾는 쿼리
    3. 비교 관점: 여러 항목을 비교하는 쿼리
    4. 문제해결 관점: 특정 문제의 해결책을 찾는 쿼리
    5. 배경 관점: 주제에 대한 배경 정보를 찾는 쿼리
    
    각 관점별로 재작성된 쿼리만 나열해주세요.
    """
    
    response = client.chat.completions.create(
        model=os.getenv("OPENAI_DEFAULT_MODEL"),
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
    )
    
    return response.choices[0].message.content.strip().split('\n')


query = "인공지능이란 무엇인가?"
print(hyperfocus_rewrite(query))

### HYDE (Hypothetical Document Embedding)

In [ ]:
from openai import OpenAI
import os

def llm_generate_hypothetical_document(original_query):
    """
    HYDE (Hypothetical Document Embedding)를 위해
    주어진 쿼리에 대한 가상의 답변(Hypothetical Document)을 생성합니다.
    """
    client = OpenAI()
    
    # HYDE 프롬프트: 쿼리에 대한 가상의 이상적인 답변을 생성하도록 지시
    prompt = f"""
    원본 쿼리: "{original_query}"
    
    위 쿼리에 대해 가장 정확하고 포괄적이며 이상적인 '가상의 답변' 또는 '가상의 문서'를 작성해주세요. 
    이 답변은 검색 증강 생성(RAG) 시스템에서 임베딩 벡터를 추출하는 데 사용될 것입니다.
    따라서, 관련 용어와 상세한 정보를 포함해야 합니다.
    
    가상의 답변만 출력하고 다른 설명은 포함하지 마세요.
    """
    
    response = client.chat.completions.create(
        model=os.getenv("OPENAI_DEFAULT_MODEL"),
        messages=[
            {"role": "system", "content": "당신은 사용자 쿼리에 대해 가상의 답변(Hypothetical Document)을 생성하는 유능한 AI입니다."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7, 
    )
    
    return response.choices[0].message.content.strip()

query = "인공지능이란 무엇인가?"
hypothetical_doc = llm_generate_hypothetical_document(query)

print(f"**원본 쿼리:** {query}\n")
print(f"**HYDE 가상의 답변:**\n{hypothetical_doc}")

### Cross Encoder

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("cross-encoder/ms-marco-MiniLM-L-6-v2")
model = AutoModelForSequenceClassification.from_pretrained("cross-encoder/ms-marco-MiniLM-L-6-v2")

query = "인공지능이란 무엇인가?"
documents = [
    "인공지능은 인간의 학습, 추론, 지각 능력을 모방하는 컴퓨터 시스템입니다.",
    "AI 기술은 머신러닝과 딥러닝을 포함합니다.",
    "자연어 처리는 컴퓨터가 인간 언어를 이해하게 합니다."
]

# 각 쿼리-문서 쌍에 대한 점수 계산
scores = []
for doc in documents:
    # 쿼리와 문서를 한 시퀀스로 결합하여 토큰화
    inputs = tokenizer(query, doc, padding=True, truncation=True, return_tensors="pt")
    
    # 점수 계산
    with torch.no_grad():
        outputs = model(**inputs)
        scores.append(outputs.logits[0].item())  # 관련성 점수 추출

# 결과 출력
for doc, score in sorted(zip(documents, scores), key=lambda x: x[1], reverse=True):
    print(f"Score: {score:.4f}, Document: {doc}")

In [ ]:
from openai import OpenAI

class LLMReranker:
    def __init__(self):
        self.client = OpenAI()
        
    def rerank(self, query, doc):
        results = []
        
        prompt = f"""
        [쿼리] {query}
        
        [문서] {doc}
        
        위 문서가 쿼리와 얼마나 관련이 있는지 0에서 10 사이의 점수로 평가해주세요.
        이유에 대한 설명 없이 숫자만 응답하세요.
        """
        
        response = self.client.chat.completions.create(
            model=os.getenv("OPENAI_DEFAULT_MODEL"),
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
        )
        
        score = float(response.choices[0].message.content.strip())
        return score
    
query = "인공지능이란 무엇인가?"
documents = [
    "인공지능은 인간의 학습, 추론, 지각 능력을 모방하는 컴퓨터 시스템입니다.",
    "AI 기술은 머신러닝과 딥러닝을 포함합니다.",
    "자연어 처리는 컴퓨터가 인간 언어를 이해하게 합니다."
]

scores = []
reranker = LLMReranker()

scores = []
for doc in documents:
    scores.append(reranker.rerank(query, doc))

for doc, score in sorted(zip(documents, scores), key=lambda x: x[1], reverse=True):
    print(f"Score: {score:.4f}, Document: {doc}")